# 로지스틱회귀 실습#2
- L1, L2정규화 및 규제화강도 조절해보기

In [1]:
import pandas as pd
import numpy as np

In [2]:
#데이터 불러오기: y값은 이미 범주형으로 되어있음.
dat_wine=pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)     # 와인 데이터 셋
dat_wine.head() # 실제 였다면 확인 작업해서 먼저 열 이름 확인 
dat_wine.columns = ['class label','alchohol','malic acid','ash','alcalinity of ash','magnesium',
                    'total phenols','flavanoids', 'nonflavanoid phenols','proanthocyanins',
                    'color intensity','hue','OD208', 'proline']  #Column names 칼럼 이름이 없어서 직접 작성해줌.
print('class label:', np.unique(dat_wine['class label']))  #Class label 출력 1, 2 3 밖에 없음.      unique는 중복된거는 출력 안함.


class label: [1 2 3]


데이터 분할

In [3]:
#전체 data를 학습과 시험 데이터셋으로 split
from sklearn.model_selection import train_test_split

X, y = dat_wine.iloc[:, 1:].values, dat_wine.iloc[:,  0].values        # x는 1번쨰 칼럼부터,  y는 0부터 가져와라   
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify = y)    # 크기는 0.3 씩 나누고 random_state 랜덤시드 값을 고정하고 stratify y 비율 유지 

print("x train:", np.shape(X_train))
print("x_test:", np.shape(X_test))
print("y train:", np.shape(y_train))
print("y_test:", np.shape(y_test))

x train: (124, 13)
x_test: (54, 13)
y train: (124,)
y_test: (54,)


In [4]:
# 표준화    # 정규화 
from sklearn.preprocessing import StandardScaler    #  StandardScaler 기본 스케일. 평균과 표준편차 사용
std = StandardScaler()
X_train_std = std.fit_transform(X_train)        # x train 값 정규화
X_test_std = std.transform(X_test)              # x test 값 정규화

### L1 정규화 L2 정규화    

규제강도 람다가 클수록 추정된 계수들의 절대값이 작아짐      

L1 정규화의 경우 규제강도가 클수록 계수에 0이 많아짐. 즉, 계수에 대응하는 특성변수를 제거하는 역할을 담당함.

In [7]:
from sklearn.linear_model import LogisticRegression     # 로지스틱 회귀

# LogisticRegression 클래스 호출 (C=1/𝜆 로 규제화의 강도를 조절)
#       C값이 작을수록 𝜆값이 커지므로 규제화 강도가 높아짐
#       C값이 클수록 𝜆값이 작아지므로 규제화 강도가 줄어듦
#       디폴트 옵션: L2정규화


#새로운 함수를 만듦
def logistic_regression(penalty, C, solver, max_iter):
    lr = LogisticRegression(penalty=penalty, C=C, solver=solver, max_iter=max_iter) # lr 즉 로지스틱 함수
    lr.fit(X_train_std, y_train)    # 로지스틱 함수를 이용한 학습

    print(f'\n---- Penalty:{penalty.upper()}, λ:{round(1/C,1)} ----'    )
    print(f'정확도(학습데이터): {round(lr.score(X_train_std, y_train),3)}')     # .score <-----  from sklearn.metrics import accuracy_score 의 정확도 계산 함수
    print(f'정확도(시험데이터): {round(lr.score(X_test_std, y_test),3)}')
    print(f'회귀계수:\n{lr.coef_}')     # 추정한 파라미터 값

In [9]:
# Logistic Regression with L2 or L1 Regularization
from sklearn.linear_model import LogisticRegression

# solver='saga' saga 방식으로 로지스틱 회귀를 학습 하겠다. 

# 패털티를 L2 정규화
logistic_regression(penalty='l2', C=10.0, solver='saga', max_iter=500) #L2 with C(=1/λ)=10
logistic_regression(penalty='l2', C=1.0, solver='saga', max_iter=500)  #L2 with C(=1/λ)=1
logistic_regression(penalty='l2', C=0.1, solver='saga', max_iter=500)  #L2 with C(=1/λ)=0.1

# 패널티를 L1 정규화 
logistic_regression(penalty='l1', C=10.0, solver='saga', max_iter=500) #L1 with C(=1/λ)=10
logistic_regression(penalty='l1', C=1.0, solver='saga', max_iter=500)  #L1 with C(=1/λ)=1
logistic_regression(penalty='l1', C=0.1, solver='saga', max_iter=500)  #L1 with C(=1/λ)=0.1

---- Penalty:L2, λ:0.1 ----
정확도(학습데이터): 1.0
정확도(시험데이터): 0.981
회귀계수:
[[ 1.23778501  0.16300691  0.30116022 -1.52835731  0.09527756  0.39429203
   0.89436941 -0.26099054  0.09993275  0.16430095  0.07762999  1.03135126
   1.35900062]
 [-1.58962339 -0.44372591 -1.27854756  1.23726174 -0.32591598 -0.57967472
   0.99618583  0.37678373  0.42467002 -1.23173538  1.17967653  0.09012622
  -1.74171491]
 [ 0.35183838  0.280719    0.97738734  0.29109557  0.23063842  0.18538269
  -1.89055525 -0.11579319 -0.52460276  1.06743443 -1.25730652 -1.12147748
   0.38271429]]
---- Penalty:L2, λ:1.0 ----
정확도(학습데이터): 1.0
정확도(시험데이터): 0.981
회귀계수:
[[ 0.75453615  0.06144845  0.23292724 -0.89225753  0.02621844  0.29450216
   0.56110416 -0.20730083  0.13472266  0.12779643  0.10222114  0.61825185
   0.90939925]
 [-0.98669337 -0.32314422 -0.65182727  0.66749859 -0.22943817 -0.20673048
   0.43845581  0.19795184  0.2445321  -0.77932743  0.63751709  0.08625316
  -1.03416851]
 [ 0.23215722  0.26169576  0.41890003  0.2247589

C:\Users\vlrms\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\linear_model\_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
C:\Users\vlrms\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\linear_model\_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
